# Практическое задание 5
# Генерация bash команды по текстовому запросу
## курс "Математические методы анализа текстов"
### ФИО:

### Постановка задачи

В этом задании вы построите систему, выдающую пользователю последовательность утилит командной строки linux (с нужными флагами) по его текстовому запросу. Вам дан набор пар текстовый запрос - команда на выходе. 

Решение этого задания будет построено на encoder-decoder архитектуре и модели transformer.


### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
* pytorch
* transformers
* sentencepiece (bpe токенизация)
* clai utils (скачать с гитхаба отсюда https://github.com/IBM/clai/tree/nlc2cmd/utils) 


### Данные

В качестве обучающей выборке используются данные, сгенерированные автоматически по запросам с сайта stack overflow. В качестве тестовых данных используются пары запросов, размеченные асессорами.

Данные можно скачать по ссылке: https://drive.google.com/file/d/1n457AAgrMwd5VbT6mGZ_rws3g2wwdEfX/view?usp=sharing

### Метрика качества

Ваш алгоритм должен выдавать пять вариантов ответа для каждого запроса. 
Для упрощения задачи метрика качества будет учитывать утилиты и флаги ответа, но не учитывать подставленные значения. Пусть $\{ u_1, \ldots, u_T \}$, $\{ f_1, \ldots, f_T \}$ --- список утилит и множества их флагов ответа алгоритма, $\{v_1, \ldots, v_T \}$, $\{ \phi_1, \ldots, \phi_T \}$ --- список утилит и множества их флагов эталонного ответа. Если ответы отличаются по длине, они дополняются `None` утилитой. 

$$ S = \frac{1}{T} \sum_{i=1}^{T} \left(\mathbb{I}[u_i = v_i]\left( 1 + \frac{1}{2}s(f_i, \phi_i)\right) - 1\right)$$

$$ s(f, \phi) = 1 + \frac{2 |f \cap \phi| - |f \cup \phi|}{\max(|f|, |\phi|)} $$

Метрика учитывает, что предсказать правильную утилиту важнее чем правильный флаг. При этом порядок флагов не важен (однако, чтобы корректно 

## Предобработка данных (2 балла)

In [ ]:
import sys
PATH_TO_CLAI_UTILS = ## YOUR CODE HERE ##
sys.path.append(PATH_TO_CLAI_UTILS)

In [ ]:
import numpy as np
import pandas as pd

from bashlint.data_tools import bash_parser, pretty_print, cmd2template
from metric.metric_utils import compute_metric
from functools import partial

from collections import Counter
import sentencepiece as spm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

Считаем данные. В столбце `invocation` находится текстовый запрос, в столбце `cmd` находится релевантная команда.

In [ ]:
train_data = pd.read_csv('data/train.csv')
train_data.head()

В тестовых данных столбец `origin` отвечает за источник данных, значения `handrafted` соответствуют парам, составленными людьми, а `mined` парам, собранным автоматически.

In [ ]:
test_data = pd.read_csv('data/test.csv')
test_data.head()

**Задание**. Проведите предобработку текста. Рекомендуется:
* перевести всё в нижний регистр
* удалить стоп-слова (специфичные для выборки)
* провести стемминг токенов
* удалить все символы кроме латинских букв

In [ ]:
def clean_text(text):
    ### YOUR CODE HERE ###

In [ ]:
train_data['text_cleaned'] = train_data['invocation'].apply(clean_text)
test_data['text_cleaned'] = test_data['invocation'].apply(clean_text)

Для обработки кода воспользуемся функцией `cmd2template`:

In [ ]:
train_data['cmd_cleaned'] = train_data['cmd'].apply(partial(cmd2template, loose_constraints=True))
test_data['cmd_cleaned'] = test_data['cmd'].apply(partial(cmd2template, loose_constraints=True))

Разделим данные на обучение и валидацию. Т.к. данных очень мало, то для валидационной выборки выделим только 100 примеров.

In [ ]:
valid_data = train_data.iloc[-100:]
train_data = train_data.iloc[:-100]

**Задание**. Стандартный формат входных данных для трансформеров — BPE токены. Воспользуйтесь библиотекой sentencepiece для обучения токенайзеров для текста и кода. Используйте небольшое число токенов.

In [ ]:
## YOUR CODE HERE ###

In [ ]:
text_tokenizer = ## YOUR CODE HERE ###
cmd_tokenizer = ## YOUR CODE HERE ###

**Задание**. Задайте датасеты и лоадеры для ваших данных. Каждая последовательность должна начинаться с BOS токена и заканчиваться EOS токеном. Рекомендуется ограничить длину входных и выходных последовательностей!

In [ ]:
PAD_ID = 0
BOS_ID = 1
EOS_ID = 2


MAX_TEXT_LENGTH = 256
MAX_CODE_LENGTH = 40

BATCH_SIZE = 64

In [ ]:
class TextToBashDataset(Dataset):
    ## YOUR CODE HERE ###

In [ ]:
train_ds = ## YOUR CODE HERE ###
valid_ds = ## YOUR CODE HERE ###

In [ ]:
loaders = {
    'train': DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True),
    'valid': DataLoader(valid_ds, batch_size=BATCH_SIZE),
}

## Обучение бейзлайна (2 балла)

In [ ]:
from transformers import BertConfig, BertModel, EncoderDecoderConfig, EncoderDecoderModel

**Задание.** Реализуйте модель encoder-decoder ниже. В качестве моделей энкодера и декодера рекомендуется использовать BertModel из библиотеки transformers, заданную через BertConfig. В случае декодера необходимо выставить параметры is_decoder=True и add_cross_attention=True. В качестве модели, <<сцепляющей>> энкодер и декодер, в одну архитектуру рекомендуется использовать EncoderDecoderModel.

**Обратите внимание!** EncoderDecoderModel поддерживает использование кэшированных результатов при последовательной генерации. Это пригодится при реализации beam-search ниже.

Для того, чтобы удобнее задавать модели, рекомендуется реализовать задание модели через конфиг. Ниже представлены базовые параметры, при которых модель должна работать быстро и с приемлемым качеством.

In [ ]:
text_model_config = {
    'vocab': text_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.1,
    'pad_id': PAD_ID,
}

cmd_model_config = {
    'vocab': cmd_tokenizer.vocab_size(),
    'hidden_size': 256,
    'num_layers': 2,
    'num_attention_heads': 8,
    'intermediate_size': 256 * 4,
    'hidden_dropout_prob': 0.1,
    'pad_id': PAD_ID,
}

In [ ]:
class TextToBashModel(nn.Module):
    def __init__(self, text_model_config, cmd_model_config):
        super(TextToBashModel, self).__init__()
        ## YOUR CODE HERE ##
        
    def forward(self, x):
        ## YOUR CODE HERE ##        

**Задание**. Обучите вашу модель ниже.

Рекомендуется:
* в качестве лосса использовать стандартную кросс-энтропию, не забывайте игнорировать PAD токены
* использовать Adam для оптимизации
* не использовать scheduler для бейзлайна (модель легко переобучается с ним)
* использовать early stopping по валидационному лоссу

In [ ]:
## YOUR CODE HERE ##

## Генерация команд (2 балла)

**Задание**. Реализуйте алгоритм beam-search в классе BeamSearchGenerator ниже. Ваша реализация должна поддерживать задание температуры софтмакса. Выходы модели, полученные на предыдущих итерациях, необходимо кэшировать для повышения скорости алгоритма. Вместо подсчёта произведения любых вероятностей необходимо считать сумму их логарифмов.

Алгоритм должен возвращать список пар из получившихся выходных последовательностей и логарифмов их вероятностей. 

In [ ]:
class BeamSearchGenerator:
    def __init__(
            self, pad_id, eos_id, bos_id,
            max_length=20, beam_width=5, temperature=1,
            device='cuda',
    ):
        """
        Parameters
        ----------
        pad_id : int
        eos_id : int
        bos_id : int
        max_length : int
            Maximum length of output sequence
        beam_width : int
            Width of the beam
        temperature : float
            Softmax temperature
        device : torch.device
            Your model device
        """
        self.pad_id = pad_id
        self.eos_id = eos_id
        self.bos_id = bos_id
        
        self.max_length = max_length
        self.beam_width = beam_width
        self.temperature = temperature
        
        self.device = device
        
    def get_result(self, model, input_text_tokens):
        """
        Parameters
        ----------
        model : TextToBashModel
        input_text_tokens : torch.tensor
            One object input tensor
        """
        ## YOUR CODE HERE ##

Протестируйте на нескольких примерах работу вашего алгоритма. Если всё реализовано правильно, то как минимум на трёх примерах из 5 всё должно работать правильно.

In [ ]:
beam_search_enginge = BeamSearchGenerator(
    pad_id=PAD_ID, eos_id=EOS_ID, bos_id=BOS_ID,
    max_length=MAX_CODE_LENGTH, beam_width=5,
    temperature=1, device='cuda',
)

In [ ]:
with torch.no_grad():
    for i in range(5):
        print()
        print('text:', valid_data.invocation.iloc[i])
        print('true:', valid_data.cmd.iloc[i])
        print('true cleaned:', valid_data.cmd_cleaned.iloc[i])

        src = valid_ds.src[i]
        pred = beam_search_enginge.get_result(model, src)
        
        scores = []
        for x, proba in pred:
            pred_cmd = cmd_tokenizer.decode(list(map(int, x)))
            score = compute_metric(pred_cmd, 1, valid_data.cmd.iloc[i])
            scores.append(score)
            print(pred_cmd, proba)
        print(max(scores))

**Задание**. Дополните функцию для подсчёта качества. Посчитайте качество вашей модели на валидационном и тестовых датасетов.

In [ ]:
def compute_all_scores(model, df, beam_engine):
    all_scores = []

    for i, (text, target_cmd) in enumerate(zip(df.text_cleaned.values, df.cmd.values)):
        input_tokens = ## YOUR CODE HERE ##
        predictions = beam_search_enginge.get_result(model, input_tokens)
        
        # get only 5 top results
        predictions = predictions[:5]
        object_scores = []
        for output_tokens, proba in predictions:
            output_cmd = cmd_tokenizer.decode(list(map(int, output_tokens)))
            score = compute_metric(output_cmd, 1, target_cmd)
            object_scores.append(score)
        
        all_scores.append(max(object_scores))
    return all_scores

Если вы всё реализовали правильно, подобрали параметры BeamSearch то ваш средний скор на валидации должен быть >= 0.25, а скор на `handcrafted` части теста >= 0.13. На `mined` части датасета скор может быть низкий, т.к. некоторых команд из датасета нет в обучении.

In [ ]:
## YOUR CODE HERE ##

## Улучшение модели (4 балла)

Вы реализовали бейзлайн, пришло время улучшить качество модели. Т.к. это последнее задание, мы не будем предлагать конкретные шаги, а только дадим несколько советов.

1. Большой источник информации о работе командной строке — её документация, man. Один из способов улучшения модели - использование мана для генерации новых примеров. Структурированный ман можно найти по ссылке https://github.com/IBM/clai/blob/nlc2cmd/docs/manpage-data.md.
2. Ещё один способ улучшить модель, разделить предсказание утилит и флагов. Т.к. задача предсказания утилит более важная, вы можете натренировать модель, которая предсказывает последовательность утилит, а затем к каждой утилите генерировать флаги.
3. Можно аугментировать данные, чтобы увеличить выборку.
4. Можно в качество входа подавать не только текстовый запрос, но и описание из мана. Т.к. всё описание достаточно большое, нужно сделать дополнительную модель, которая будет выбирать команды, для которых нужно вытащить описание.
5. Найти дополнительные данные, улучшающие обучение
6. Как всегда можно просто сделать больше слоёв, увеличить размер скрытого слоя и т.д.

От вас ожидается скор на `mined` >= 0 при скоре на `handrafted` >= 0.16.

In [ ]:
## YOUR CODE HERE ##

## Бонусные баллы (до 3 баллов)

При существенном улучшении качества будут назначаться бонусные баллы. На тестовых датасетах реально выбить качество >= 0.3 на каждом, но усилий потребуется немало...